In [ ]:
# you may need to checkout the branch tickets/SITCOM-1752 from ts_m1m3_utils
from lsst.ts.m1m3.utils.dynamic_test_analysis import load_config, run_single_dynamic_test, create_color_dict, SlewPlotter

# Configuration
Below is a configuration dictionary, for each dynamic test provide begin and end times as well as the block info (label to be used for plotting, I have been doing speed settings plus m1m3 setting e.g `100% GGGG`).
A dayObs and seqNum range may be provided instead of timestamps

In [ ]:
dynamic_test_info_dict = {
    "20241201_T293_1": {
        "begin_time": "2024-12-02 23:34:11",
        "end_time": "2024-12-02 23:49:52",
        "block_info": "20% GGGG",
        "block": "T293",
        
    },
    "20241130_T293_6":{
        "begin_time": "2024-12-01 08:49:07",
        "end_time": "2024-12-01 09:02:06",
        "block_info": "20% GGGR",
        "block": "T293"
    },
    "20241130_T293_4":{
        "begin_time": "2024-11-30 22:49:52",
        "end_time": "2024-11-30 23:03:17",
        "block_info": "20% GGRR",
        "block": "T293"
    },
    "20241205_T293_1":{
        "begin_time": "2024-12-06 08:52:34",
        "end_time": "2024-12-06 09:05:10",
        "block_info": "20% GGGG J=1",
        "block": "T293"
    },
     "20241205_T293_2":{
        "begin_time": "2024-12-06 09:13:08",
        "end_time": "2024-12-06 09:25:10",
        "block_info": "20% GGGG J=1 a=1",
        "block": "T293"
    },
}

The load config function will provide a default configuration. 
Additionally, a config_file may be provided as a string to a yaml file with configuration. 
Finally any kwargs provided will superceed the default or file. 
Config options and default values:
```
{'begin_seq_num': None, 
 'end_seq_num': None,
 'begin_time': None,
 'end_time': None,
 'day_obs': None,
 'block_info': None,
 'block': None,
 'tmax': None,
 'tmin': None,
 'exclude_list': [],
 'data_dir': './data/',
 'plot_dir': './plots/',
 'save_data': True,
 'make_plots': True,
 'hp_col_keys': [],
 'act_groups': ['all',
  'quadrant_1',
  'quadrant_2',
  'quadrant_3',
  'quadrant_4',
  'orientation_Y_PLUS',
  'orientation_Y_MINUS',
  'orientation_X_PLUS',
  'orientation_X_MINUS'],
 'act_types': ['primary', 'secondary'],
 'fa_col_keys': []}
```

# Retrieving all of the data
The `run_single_dynamic_test` takes the config as kwargs and returns:
- a dataframe with stats on each slew including the slew state.
- a dataframe with hardpoint telemetry for each slew
- a dataframe with forceactuator following errors for each slew

if `save_data=True` these dataframes will be saved in the `data_dir`.
if `make_plots=True` single test plots will be made for every column in hp_col_keys and fa_col_keys (there is some automatic population of fa_col_keys in the config)


stats_frame columns:
`'seq_num', 'el_start', 'el_end', 'el_distance', 'az_start', 'az_end','az_distance', 'total_distance', 'max_hp_force', 'min_hp_force', 'max_fa_following_error', 'min_fa_following_error', 'day_obs', 'block','block_info', 'state'`
The other data_frames columns will list out at the end of this notebook

In [ ]:
dynamic_test_data_dict = {}
for test in dynamic_test_info_dict:
    config = load_config(**dynamic_test_info_dict[test], save_data=False, make_plots=False)
    stats_frame, hp_forces_df, fa_following_errors_df = await run_single_dynamic_test(**config)
    dynamic_test_data_dict[test]={}
    dynamic_test_data_dict[test]["stats_frame"] = stats_frame
    dynamic_test_data_dict[test]["hp_forces_df"] = hp_forces_df
    dynamic_test_data_dict[test]["fa_following_errors_df"] = fa_following_errors_df

# if you get an asyncio error, in a notebook, try running again

# Options for plotting 
## Hardpoint Forces:
- `'measuredForce0'`
- `'measuredForce1'`
- `'measuredForce2'`
- `'measuredForce3'`
- `'measuredForce4'`
- `'measuredForce5'`
- `'fx'`
- `'fy'`
- `'fz'`
- `'mx'`
- `'my'`
- `'mz'`
## Force Actuator Following Errors
- Any of the primary/secondary following errors (e.g. `'primaryCylinderFollowingError27'` or `'secondaryCylinderFollowingError106'` )


### stats computed on groups of force actuators:
The naming format is `'{group}_{act_type}_{stat}`'

- extrema (max will plot max & min): `'all_primary_max_val'`

- median + 1 sigma confidence interval: `'all_primary_median_val'`

- abs(max): `'all_primary_absmax_val'`

- std: `'all_primary_std_val'`

### groups that have been split out:

- All actuators: `'all_primary_max_val'`
- Orientation `["X_PLUS","X_MINUS", "Y_PLUS","Y_MINUS"]`: `'orientation_X_MINUS_primary_max_val'`
- Quadrant \[1,2,3,4\]: `''quadrant_1_secondary_max_val'`'


# Comparing dynamic tests
The SlewPlotter class will return an 8 panel plot that groups each of the slews by type (so they should have similar profiles) and aligns the slews from the slew start or slew stop (again to facilitate comparison) 

In [ ]:
color_dict = create_color_dict(dynamic_test_info_dict.keys(), 'tab10')
plot_dir = None # switch to a directory to save plots
sp=SlewPlotter(plot_dir=plot_dir)
for col in ['all_primary_max_val','all_primary_median_val']:#,'all_primary_absmax_val','all_primary_std_val']:
    fig=sp.multi_test_plot(
        col_key=col,
        dynamic_test_data_dict=dynamic_test_data_dict, 
        dynamic_test_info_dict=dynamic_test_info_dict,
        color_dict=color_dict,
        time_align="stop",
        duration=6,
        # xmin=-0.2,
        # xmax=0,
    )
    fig

In [ ]:
for i in range(1,5):
    for col in [f'quadrant_{i}_primary_absmax_val']:
        fig=sp.multi_test_plot(
            col_key=col,
            dynamic_test_data_dict=dynamic_test_data_dict, 
            dynamic_test_info_dict=dynamic_test_info_dict,
            color_dict=color_dict,
            time_align="start",
            duration=1,
            # xmin=0,
            # xmax=1.1,
        )
        fig

In [ ]:
orientation=["X_PLUS","X_MINUS", "Y_PLUS","Y_MINUS"]
for j in orientation:
    for col in [f'orientation_{j}_primary_absmax_val']:
        fig=sp.multi_test_plot(
            col_key=col,
            dynamic_test_data_dict=dynamic_test_data_dict, 
            dynamic_test_info_dict=dynamic_test_info_dict,
            color_dict=color_dict,
            time_align="start",
            duration=1,
            # xmin=0,
            # xmax=1.1,
        )
        fig

# Plot slews for a single dynamic test
Also loading a config from a yaml file

In [ ]:
config_file = "./dynamical_test_configs/20241201_T293_1.yaml"
config = load_config(config_file=config_file, save_data=False, make_plots=False)

In [ ]:
stats_frame, hp_forces_df, fa_following_errors_df = await run_single_dynamic_test(**config)

In [ ]:
test = "20241205_T293_1"
plot_dir = None # switch to a directory to save plots
sp=SlewPlotter(plot_dir=plot_dir)
fig=sp.single_test_plot(
    col_key='all_secondary_absmax_val',
    stats_frame=stats_frame,
    hp_forces_df=hp_forces_df, 
    fa_following_errors_df = fa_following_errors_df,
    time_align="start",
    duration=1,
    block="T293",
    block_info="",
    xmin=None,
    xmax=None,
    day_obs=None,
)
if plot_dir is not None:
    fig

# List out all columns

In [ ]:
dynamic_test_data_dict[test]["hp_forces_df"].columns.values

In [ ]:
dynamic_test_data_dict[test]["fa_following_errors_df"].columns.values